In [1]:
import boto3
from dotenv import load_dotenv
import os
from openai import OpenAI, AuthenticationError
from botocore.exceptions import NoCredentialsError, ClientError
import re
import json

In [2]:
load_dotenv()

True

In [3]:
# openai connection 
api_key = os.environ["OPENAI_KEY"]

def create_client(api_key):
    try:
        client = OpenAI(api_key=api_key)
        client.models.list()
        return client
    except AuthenticationError:
        print("Incorrect API")
    return None

client = create_client(api_key)

In [4]:
bucket_name = 's3-ai-agent'
s3 = boto3.client("s3")

In [7]:

def list_files(prefix: str = ""):
    try:
        resp = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)
        if "Contents" not in resp:
            return []
        return [o["Key"] for o in resp["Contents"]]
    except ClientError as e:
        return [f"Error: {e}"]


def read_file(key: str):
    try:
        obj = s3.get_object(Bucket=bucket_name, Key=key)
        text = obj["Body"].read().decode("utf-8", errors="replace")
        return text[:4000]
    except ClientError as e:
        return f"Error: {e}"


def s3_agent(prompt: str, history: list):
    tools = [
        {
            "type": "function",
            "function": {
                "name": "list_files",
                "description": "List files in the fixed S3 bucket. Optional folder prefix.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "prefix": {"type": "string", "description": "Folder path like 'data/'."}
                    },
                    "required": [],
                },
            },
        },
        {
            "type": "function",
            "function": {
                "name": "read_file",
                "description": "Read text content of a specific file in the fixed S3 bucket.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "key": {"type": "string", "description": "File key like 'data/report.txt'."}
                    },
                    "required": ["key"],
                },
            },
        },
    ]

    if not any(m["role"] == "system" for m in history):
        history.insert(
            0,
            {
                "role": "system",
                "content": (
                    f"You are an AI agent with read-only access to the S3 bucket '{bucket_name}'. "
                    "You can list and read files. Use tools when needed. "
                    "Keep the conversation context to provide coherent responses."
                ),
            },
        )

    history.append({"role": "user", "content": prompt})

    response = client.chat.completions.create(
        model="gpt-4.1",
        messages=history,
        tools=tools,
        tool_choice="auto",
    )

    msg = response.choices[0].message

    if getattr(msg, "tool_calls", None):
        tool = msg.tool_calls[0]
        name = tool.function.name
        raw_args = tool.function.arguments

        try:
            args = json.loads(raw_args) if isinstance(raw_args, str) else (raw_args or {})
        except Exception:
            args = {}

        if name == "list_files":
            result = list_files(**args)
            result_text = "\n".join(result) if isinstance(result, list) else str(result)
            tool_context = f"Files (prefix={args.get('prefix', '')}):\n{result_text}"
        elif name == "read_file":
            result = read_file(**args)
            tool_context = f"File {args.get('key', '')} content:\n{result}"
        else:
            tool_context = f"Unknown tool: {name}"

        history.append({"role": "assistant", "content": f"[Tool result {name}]\n{tool_context}"})
        history.append({"role": "user", "content": "Continue your response based on this result."})

        follow = client.chat.completions.create(model="gpt-5", messages=history)
        follow_msg = follow.choices[0].message
        answer = (follow_msg.content or "").strip()
        history.append({"role": "assistant", "content": answer})
        return answer, history

    answer = (msg.content or "").strip()
    history.append({"role": "assistant", "content": answer})
    return answer, history


if __name__ == "__main__":
    print(f"S3 Agent (read-only) – bucket: {bucket_name}")
    conversation = []

    while True:
        user_input = input("You: ")
        if user_input.lower() in ("exit", "quit"):
            break
        try:
            reply, conversation = s3_agent(user_input, conversation)
            print("Agent:", reply)
        except Exception as e:
            print("Error:", e)

S3 Agent (read-only) – bucket: s3-ai-agent


You:  list availble files


Agent: I see two files in the bucket:
- customer_feedback_Q3_2025.pdf
- customer_feedback_Q3_2025.txt

I’ll start by opening the TXT for a quick summary, sentiment/themes, and key action items. If you prefer the PDF instead, let me know.

Proceeding to read: customer_feedback_Q3_2025.txt.


You:  base on txt file tell me what i can do or improve to make my service better, what are clients compaining about?


Agent: Top complaints from the TXT:
- Billing issues: duplicate charges, unclear invoices/charges, confusing refund process.
- Refunds: hard to request, unclear timelines.
- Onboarding for alerts: dashboard looks good but users get lost setting up alerts.

What to improve (prioritized, actionable):
1) Fix billing pain quickly
- Prevent double charges: enforce idempotency keys on payment/renewal calls, de-dupe webhook events, tighten retry logic.
- Make billing clear in-app: invoice breakdown (plan, proration, taxes, discounts), upcoming charges, next renewal date, “Why was I charged?” explainer next to each line item.
- Improve emails: plain-language invoice and payment-failure emails with cause, next steps, and links to billing page.
- SLA for billing tickets: fast lane triage (e.g., 4 business hours first response, 48 hours resolution), dedicated “Billing” queue.

2) Simplify refunds and communicate timelines
- Self-serve refund request flow: pick charge, select reason, auto-calc eli

You:  can you show me this file. i mean can you write its content 1:1


Agent: Certainly! Here is the exact 1:1 content of the file customer_feedback_Q3_2025.txt:

---
Customer Feedback Report - Q3 2025

Overview:
This report summarizes customer satisfaction feedback collected from survey forms, chat transcripts, and post-purchase reviews.

Highlights:
- Overall satisfaction increased compared to Q2.
- Customers appreciate the faster response time and improved dashboard UI.
- Main area for improvement: billing clarity and refund policy communication.

Individual Feedback Samples:
1. "Great experience overall! The support agent was quick and solved my issue within minutes."
   Rating: 5/5
   Category: Support

2. "The new dashboard is beautiful, but I got lost trying to set up alerts."
   Rating: 4/5
   Category: Product UI

3. "I was charged twice for my subscription and it took a week to fix it."
   Rating: 2/5
   Category: Billing

4. "Love the new analytics section. Charts load fast, and I can export data easily."
   Rating: 5/5
   Category: Product Per

You:  exit
